In [1]:
from lasertram import LaserTRAM, LaserCalc
import lasertram as lt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import mendeleev
def oxide_to_ppm(wt_percent, int_std):
    """
    convert concentration internal standard analyte oxide in weight percent to
    concentration ppm for a 1D series of data

    Parameters
    ----------
    wt_percent : array-like
        the oxide values to be converted to ppm
    int_std : string
        the internal standard used in the experiment (e.g., '29Si', '43Ca',
                                                      '47Ti')
        currently supported elements:

            - 'SiO2'
            - 'TiO2'
            - 'Al2O3'
            - 'Cr2O3'
            - 'MnO'
            - 'FeO'
            - 'K2O'
            - 'CaO'
            - 'Na2O'
            - 'NiO'
            - 'MgO'

    Returns
    -------
    ppm : array-like
        concentrations in ppm the same shape as the wt_percent input

    """

    el = [i for i in int_std if not i.isdigit()]

    if len(el) == 2:
        element = el[0] + el[1]

    else:
        element = el[0]

    oxides = [
        "SiO2",
        "TiO2",
        "Al2O3",
        "Cr2O3",
        "MnO",
        "FeO",
        "K2O",
        "CaO",
        "Na2O",
        "NiO",
        "MgO",
    ]

    for o in oxides:
        if element in o:
            oxide = o

    s = oxide.split("O")
    cat_subscript = s[0]
    an_subscript = s[1]

    cat_subscript = [i for i in cat_subscript if i.isdigit()]
    if cat_subscript:
        cat_subscript = int(cat_subscript[0])
    else:
        cat_subscript = 1

    an_subscript = [i for i in an_subscript if i.isdigit()]
    if an_subscript:
        an_subscript = int(an_subscript[0])
    else:
        an_subscript = 1

    ppm = 1e4 * (
        (wt_percent * mendeleev.element(element).atomic_weight * cat_subscript)
        / (
            mendeleev.element(element).atomic_weight
            + mendeleev.element("O").atomic_weight * an_subscript
        )
    )
    return ppm


In [2]:
data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\spot_test_timestamp_raw_data.xlsx").set_index('SampleLabel')
data.head()

,timestamp,Time,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
SampleLabel,,,,,,,,,,,,,,,
BCR-2G_1,2021-03-01 22:08:14,13.24,500.010000,0.0,100.0004,46285.535670,100.0004,0.0000,600.014400,0.0,0.0,0.0,0.0,0.0,100.0004
BCR-2G_1,2021-03-01 22:08:14,163.60,700.019601,0.0,200.0016,43274.778818,0.0000,200.0016,900.032401,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,313.96,600.014400,0.0,400.0064,46586.651171,300.0036,200.0016,1000.040002,0.0,0.0,0.0,0.0,0.0,200.0016
BCR-2G_1,2021-03-01 22:08:14,464.23,800.025601,0.0,100.0004,45683.326387,100.0004,100.0004,1400.078404,0.0,0.0,0.0,0.0,0.0,0.0000
BCR-2G_1,2021-03-01 22:08:14,614.50,600.014400,0.0,300.0036,46185.165445,200.0016,200.0016,700.019601,0.0,0.0,0.0,0.0,0.0,0.0000


In [3]:
samples = data.index.unique().dropna().tolist()

sample = samples[0]

bkgd_interval = (5,10)
keep_interval = (20,50)

spot = LaserTRAM(name = sample)
spot.get_data(data.loc[sample,:])
spot.assign_int_std("29Si")
spot.assign_intervals(bkgd_interval, keep_interval)
spot.get_bkgd_data()
spot.subtract_bkgd()
spot.get_detection_limits()
spot.subtract_bkgd()
spot.normalize_interval()
spot.make_output_report()

In [4]:
spot = LaserTRAM(name = sample)
lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = (5,10),
        keep = (20,50),
        internal_std = "29Si",
        despike = False,
        output_report = True
        )

In [5]:
bkgd_interval = (5,8)
keep_interval = (25,45)

int_std = "29Si"

my_spots = []
print("PROCESSING SPOTS\n")
for sample in tqdm(samples):
    spot = LaserTRAM(name = sample)
    lt.process_spot(
        spot,
        raw_data = data.loc[sample,:],
        bkgd = bkgd_interval,
        keep = keep_interval,
        internal_std = int_std,
        despike = False,
        output_report = True
        )
    my_spots.append(spot)

print("COMBINING OUTPUT REPORTS\n")
processed_df = pd.DataFrame()
for spot in tqdm(my_spots):
    processed_df = pd.concat([processed_df,spot.output_report])


PROCESSING SPOTS



  0%|          | 0/40 [00:00<?, ?it/s]

COMBINING OUTPUT REPORTS



  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
srm_data = pd.read_excel(r"C:\Users\jlubbers\OneDrive - DOI\Research\Coding\laserTRAM-DB\tests\laicpms_stds_tidy.xlsx")


In [10]:
concentrations = LaserCalc(name="experiment_test")
concentrations.get_SRM_comps(srm_data)
concentrations.get_data(processed_df)
concentrations.set_calibration_standard("BCR-2G")
concentrations.drift_check()
concentrations.get_calibration_std_ratios()
concentrations.set_internal_standard_concentrations(
    concentrations.data["Spot"],
    np.full(concentrations.data["Spot"].shape[0], 71.9),
    np.full(concentrations.data["Spot"].shape[0], 1),
)
concentrations.calculate_concentrations()



In [11]:
concentrations.unknown_concentrations

,Spot,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
sample,,,,,,,,,,,,,,
unknown,unknown_nist-612_1,37.430614,66.744241,11253.642287,336086.996322,92854.309550,462.987407,52.753570,88.168794,41.898719,41.422178,41.744164,40.657770,39.096578
unknown,unknown_nist-612_2,38.075967,64.903258,11200.470510,336086.996322,92529.236470,461.701655,55.730132,87.870494,41.842380,41.045609,41.710204,40.602943,38.615838
unknown,unknown_nist-612_3,38.387653,67.928245,11301.420221,336086.996322,95362.177057,480.710728,53.898815,90.111601,42.969609,42.754045,42.572354,41.338098,39.761614
unknown,unknown_nist-612_4,36.177737,67.208294,11431.675708,336086.996322,93803.808039,475.349004,51.048630,89.440333,42.985408,41.783537,41.769527,41.219493,39.669591
unknown,unknown_nist-612_5,36.275922,68.004839,11303.805177,336086.996322,93664.065054,472.508925,49.301551,88.764919,42.267666,40.808451,41.076491,40.852124,39.374232


In [12]:
concentrations.SRM_concentrations

,Spot,7Li,24Mg,27Al,29Si,43Ca,48Ti,57Fe,88Sr,138Ba,139La,140Ce,153Eu,208Pb
sample,,,,,,,,,,,,,,
ATHO-G,ATHO-G_1,25.980831,580.427382,65935.976659,353403.141361,12397.257964,1533.414628,23709.618969,96.453399,551.933349,58.536507,123.376056,2.782576,5.870519
ATHO-G,ATHO-G_2,26.561203,576.232696,66035.491116,353403.141361,12258.604245,1553.972598,23894.660345,97.302222,550.278540,57.879504,123.757767,2.838512,5.711188
ATHO-G,ATHO-G_3,26.130965,593.744416,66729.776409,353403.141361,12713.028897,1587.795506,24336.310933,100.168373,566.023343,60.300028,127.906740,2.885872,5.821716
ATHO-G,ATHO-G_4,26.456721,589.486241,66608.572814,353403.141361,12346.345537,1583.271829,24474.287367,99.675454,569.880854,59.725933,126.627102,2.915028,5.829750
ATHO-G,ATHO-G_5,26.844400,602.087995,67327.098143,353403.141361,12748.827478,1611.208351,24472.469318,101.061670,578.113135,60.679155,128.987875,2.923042,6.079639
ATHO-G,ATHO-G_6,27.287583,598.798143,66844.316139,353403.141361,12560.027700,1610.955995,24817.289420,100.505539,574.435511,60.511540,129.612144,2.936821,6.061200
ATHO-G,ATHO-G_7,27.412277,604.449042,67716.076949,353403.141361,12803.576327,1634.125039,24907.019940,102.234365,581.584600,60.907366,130.703431,2.974678,6.036017
ATHO-G,ATHO-G_8,26.770341,596.515839,67008.093098,353403.141361,12612.207319,1632.880971,24843.757656,101.078301,575.481662,60.934435,130.304118,2.956797,6.024279
ATHO-G,ATHO-G_9,26.978145,600.452897,67009.680987,353403.141361,12610.610941,1625.498113,24724.468250,100.598588,571.261028,60.561577,128.270192,2.831381,5.882048
